# Explore OpenAI function calling interface.
I want to test the use of function calls.

First we use openai api directly.

In [1]:
import langchain
import openai
import json

import os

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as infile:
        return infile.read()

openai.api_key = open_file('../OpenAIChat/key_openai.txt').strip()
os.environ["OPENAI_API_KEY"] = openai.api_key


In [2]:
function_descriptions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "description": "The temperature unit to use. Infer this from the users location.",
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["location", "unit"],
        },
    }
]

In [3]:
user_query = "What's the weather like in Sydney?"
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        
        # This is the chat message from the user
        messages=[{"role": "user", "content": user_query}],
    
        
        functions=function_descriptions,
        function_call="auto",
    )
ai_response_message = response["choices"][0]["message"]
print(ai_response_message)

{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"Sydney\",\n  \"unit\": \"celsius\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}


Now the data is a string. We can use eval to make it into objects.

In [4]:
user_location = eval(ai_response_message['function_call']['arguments']).get("location")
user_unit = eval(ai_response_message['function_call']['arguments']).get("unit")


Do a dummy function.

In [5]:
def get_current_weather(location, unit):
    
    """Get the current weather in a given location"""
    
    weather_info = {
        "location": location,
        "temperature": "22",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

function_response = get_current_weather(
    location=user_location,
    unit=user_unit,
)


In [6]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=[
        {"role": "user", "content": user_query},
        ai_response_message,
        {
            "role": "function",
            "name": "get_current_weather",
            "content": function_response,
        },
    ],
)
print (second_response['choices'][0]['message']['content'])

The weather in Sydney is currently sunny and windy, with a temperature of 22 degrees Celsius.


Now lets make the calls using langchain.

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage

llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

Do exactly the same as before but with langchain objects.

In [8]:
first_response = llm.predict_messages([HumanMessage(content=user_query)],
                                      functions=function_descriptions)
first_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Sydney",\n  "unit": "celsius"\n}'}}, example=False)

In [9]:
user_location = eval(first_response.additional_kwargs['function_call']['arguments']).get("location")
user_unit = eval(first_response.additional_kwargs['function_call']['arguments']).get("unit")
function_name = first_response.additional_kwargs["function_call"]["name"]

function_response = get_current_weather(
    location=user_location,
    unit=user_unit,
)


In [10]:
second_response = llm.predict_messages([HumanMessage(content=user_query),
                                        AIMessage(content=str(first_response.additional_kwargs)),
                                        ChatMessage(role='function',
                                                    additional_kwargs = {'name': function_name},
                                                    content = function_response
                                                   )
                                       ],
                                       functions=function_descriptions)

In [11]:
second_response.content

'The weather in Sydney is currently 22 degrees Celsius and it is sunny with windy conditions.'

Now I want to be able to state a function that has now parameters. Lets experiment.

In [12]:
import datetime

def get_time_date():
    return datetime.datetime.now().isoformat()

get_time_date()

'2023-08-29T17:00:04.486519'

In [13]:
function_descriptions.append(
    {
        "name": "get_time_date",
        "description": "Get the current date and time in ISO format",
        "parameters": {
            "type": "object",
            "properties": {}
        },
    }
)

In [14]:
first_response = llm.predict_messages([HumanMessage(content="What is the current time and date?")],
                                      functions=function_descriptions)
first_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_time_date', 'arguments': '{}'}}, example=False)

In [15]:
second_response = llm.predict_messages([HumanMessage(content="what is the current time and date formatted as hh:mm dd-mm-yyyy?"),
                                        AIMessage(content=str(first_response.additional_kwargs)),
                                        ChatMessage(role='function',
                                                    additional_kwargs = {'name': function_name},
                                                    content = get_time_date()
                                                   )
                                       ],
                                       functions=function_descriptions)

second_response.content

'The current time and date formatted as hh:mm dd-mm-yyyy is 17:00 29-08-2023.'